In [ ]:
from amharic_cleaner import AmharicCleaner

def preprocess_text(message):
    # Remove unwanted characters and normalize text
    cleaner = AmharicCleaner()
    message = cleaner.remove_punct(message)  # Removes punctuation
    tokens = cleaner.tokenize(message)       # Tokenize into words
    normalized_tokens = cleaner.normalize_tokens(tokens)  # Normalize tokens
    return normalized_tokens

# Apply the preprocessing to your data
df['cleaned_message'] = df['message'].apply(preprocess_text)
print(df[['message', 'cleaned_message']])
